In [1]:
import os
import random

import torch
import torch.nn as nn
import torch.optim as optim

import time
import numpy as np

from FaceLandmarkDetection.src.detection.model.resnet import resnet18
from FaceLandmarkDetection.src.detection.data.dataset import FaceLandmarksDataset
from FaceLandmarkDetection.src.detection.data.prepare_data import Transforms

In [2]:
from FaceLandmarkDetection.config import data_dir, data_file

In [8]:
data_dir = 'FaceLandMarkDetection/data/face_landmark_dataset'
data_file = 'FaceLandMarkDetection/data/face_landmark_dataset/labels_ibug_300W_train.xml'

In [9]:
transformed_dataset = FaceLandmarksDataset(data_file=data_file, data_dir=data_dir, transform=Transforms())

In [10]:
len(transformed_dataset)

6666

In [28]:
def set_random_seeds(random_seed=0):
    torch.manual_seed(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)

    
def get_data():
    transformed_dataset = FaceLandmarksDataset(data_file=data_file, data_dir=data_dir, transform=Transforms())
    toy_factor = 1/6
    toy_dataset, other_dataset = torch.utils.data.random_split(transformed_dataset,
                                              [int(len(transformed_dataset)*toy_factor), int(len(transformed_dataset)*(1-toy_factor))])
    # split the dataset into validation and test sets
    print(len(toy_dataset))
    len_valid_set = int(0.1 * len(toy_dataset))
    len_train_set = len(toy_dataset) - len_valid_set
    train_dataset, valid_dataset = torch.utils.data.random_split(toy_dataset, [len_train_set, len_valid_set])
    return train_dataset, valid_dataset


def make_loader(dataset, batch_size):
    loader = torch.utils.data.DataLoader(dataset=dataset,
                                         batch_size=batch_size,
                                         shuffle=True,
                                         num_workers=2)
    return loader


def evaluate_model(model, test_loader, device, criterion=None):
    model.eval()
    model.to(device)

    running_loss = 0

    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        labels = labels.view(labels.size(0), -1).float()
        outputs = model(inputs)

        if criterion is not None:
            loss = criterion(outputs, labels).item()
        else:
            loss = 0

        # statistics
        running_loss += loss * inputs.size(0)
        
    eval_loss = running_loss / len(test_loader.dataset)
    return eval_loss

def train_model(model, train_loader, test_loader, device, num_epochs):
    learning_rate = 1e-2

    criterion = nn.MSELoss()

    model.to(device)
    
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        # Training
        model.train()

        running_loss = 0
        running_corrects = 0
        # pruner.update_epoch(epoch)
        for inputs, labels in train_loader:

            inputs = inputs.to(device)
            labels = labels.to(device)
            labels = labels.view(labels.size(0), -1).float()
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # statistics
            running_loss += loss.item() * inputs.size(0)

        train_loss = running_loss / len(train_loader.dataset)

        # Evaluation
        model.eval()
        eval_loss = evaluate_model(model=model, test_loader=test_loader, device=device, criterion=criterion)
        print("Epoch: {:02d} Train Loss: {:.3f} Eval Loss: {:.3f}".format(epoch, train_loss, eval_loss))

    return model

def calibrate_model(model, loader, device=torch.device("cpu:0")):
    model.to(device)
    model.eval()
    for inputs, labels in loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        labels = labels.view(labels.size(0), -1).float()
        _ = model(inputs)

def measure_inference_latency(model, device, input_size=(1,1,224,224), num_samples=100):
    model.to(device)
    model.eval()
    start_time = time.time()
    for _ in range(num_samples):
        x = torch.rand(size=input_size).to(device)
        _ = model(x)
    end_time = time.time()
    elapsed_time = end_time - start_time
    elapsed_time_ave = elapsed_time / num_samples
    return elapsed_time_ave

def save_model(model, model_dir, model_filename):
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    model_filepath = os.path.join(model_dir, model_filename)
    torch.save(model.state_dict(), model_filepath)

def load_model(model, model_filepath, device):
    model.load_state_dict(torch.load(model_filepath, map_location=lambda storage, loc: storage))
    return model

def save_torchscript_model(model, model_dir, model_filename):
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    model_filepath = os.path.join(model_dir, model_filename)
    torch.jit.save(torch.jit.script(model), model_filepath)

def load_torchscript_model(model_filepath, device):
    model = torch.jit.load(model_filepath, map_location=device)
    return model

def create_model(num_classes=10):
    model = resnet18(pretrained=True)

    # We would use the pretrained ResNet18 as a feature extractor.
    for param in model.parameters():
        param.requires_grad = False
    
    # Modify the input channels
    model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
    
    # Modify the last FC layer
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, num_classes)
    return model

In [19]:
random_seed = 0
num_classes = 136
cuda_device = torch.device("cuda:0")
cpu_device = torch.device("cpu:0")

model_dir = "saved_models"
model_filename = "resnet18_FLM.pt"
model_filepath = os.path.join(model_dir, model_filename)
set_random_seeds(random_seed=random_seed)

In [29]:
# Create an untrained model.
model = create_model(num_classes=num_classes)
train_dataset, val_dataset = get_data()
train_loader = make_loader(train_dataset, 64)
val_loader = make_loader(val_dataset, 32)

1111


In [30]:
model = train_model(model=model, train_loader=train_loader, test_loader=val_loader, device=cpu_device, num_epochs=10)

Epoch: 00 Train Loss: 3.717 Eval Loss: 1.787
Epoch: 01 Train Loss: 0.651 Eval Loss: 0.451
Epoch: 02 Train Loss: 0.196 Eval Loss: 0.141
Epoch: 03 Train Loss: 0.083 Eval Loss: 0.064
Epoch: 04 Train Loss: 0.052 Eval Loss: 0.048
Epoch: 05 Train Loss: 0.041 Eval Loss: 0.039
Epoch: 06 Train Loss: 0.034 Eval Loss: 0.035
Epoch: 07 Train Loss: 0.029 Eval Loss: 0.030
Epoch: 08 Train Loss: 0.026 Eval Loss: 0.027
Epoch: 09 Train Loss: 0.023 Eval Loss: 0.024


In [ ]:
# Save model.
save_model(model=model, model_dir=model_dir, model_filename=model_filename)